In [1]:
import pandas as pd
import sqlalchemy as sql
import matplotlib.pyplot
import numpy

### NOTE: You will need to replicate the OONI db

Please see [our instructions for replicating the OONI DB on Amazon EC2](https://gist.github.com/elsehow/7e26f4949cb8fca3bf80b5de5be7f106).

Once you've done that, you can run this code on your EC2 instance

In [7]:
sql_engine = sql.create_engine('postgresql:///metadb')

# Compute interference rate for 2019

First, we find all reports for 2019.

In [16]:
reports_2019 = pd.read_sql_query("SELECT * from report where to_char(report.test_start_time, 'YYYY') = '2019'", sql_engine)

In [21]:
total_reports_2019 = len(reports_2019)
total_reports_2019

3940141

Next, we'll find the total number of reports made in each country. This will allow us to compute our rates

In [20]:
cc_by_num_reports_2019 = reports_2019.groupby("probe_cc").nunique()

In [25]:
cc_by_num_reports_2019 = cc_by_num_reports_2019['report_no']

In [28]:
# SANITY CHECK:
# number of reports across all countries should equal the total number of reports
assert cc_by_num_reports_2019.sum() == total_reports_2019

Now, we'll find the number of *confirmed* and *anomolous* interference events in each country.

In [40]:
# some sample queries to help us see the schema...
pd.read_sql_query("SELECT * from ooexpl_wc_input_counts LIMIT 3", sql_engine)

,input,confirmed_count,anomaly_count,failure_count,total_count,test_day,bucket_date,probe_cc,probe_asn
0,https://www.sendspace.com/,0,0,0,1,2019-08-24,2019-08-24,US,701
1,https://www.sendspace.com/,0,0,0,2,2019-08-24,2019-08-24,US,7018
2,https://www.sendspace.com/,0,0,0,3,2019-08-24,2019-08-24,US,7922


In [32]:
probe_results_2019 = pd.read_sql_query("SELECT * from ooexpl_wc_input_counts where to_char(ooexpl_wc_input_counts.test_day, 'YYYY') = '2019'", sql_engine)

In [39]:
cc_by_probe_results_2019 = probe_results_2019.groupby('probe_cc').nunique()
cc_by_probe_results_2019.head()

,input,confirmed_count,anomaly_count,failure_count,total_count,test_day,bucket_date,probe_cc,probe_asn
probe_cc,,,,,,,,,
AD,38,1,1,1,1,1,1,1,1
AE,1610,3,1,1,3,59,60,1,6
AF,168,1,1,1,1,5,5,1,4
AG,28,1,1,1,1,1,1,1,1
AL,590,1,1,1,2,19,19,1,10


We'll now merge the probe results with the number of reports, so that we can compute the rates of confirmed/anomolous reports for each country.

In [55]:
df = cc_by_probe_results_2019[['confirmed_count', 'anomaly_count']]

In [60]:
interference_measurements_by_country=\
pd.DataFrame({'probe_cc': cc_by_num_reports_2019.index,
             'total': cc_by_num_reports_2019.values})\
    .merge(df, how='outer', left_on='probe_cc', right_on='probe_cc')

In [61]:
interference_measurements_by_country.head()

,probe_cc,total,confirmed_count,anomaly_count
0,AD,75,1.0,1.0
1,AE,6826,3.0,1.0
2,AF,1551,1.0,1.0
3,AG,48,1.0,1.0
4,AI,19,NaN,NaN


In [62]:
interference_measurements_by_country.to_csv('interference-measurements-by-country-2019.csv')

# Lily's work

In [8]:
first_10_meas = pd.read_sql_query("SELECT * from ooexpl_wc_input_counts LIMIT 10", sql_engine)
first_10_meas.head()

,input,confirmed_count,anomaly_count,failure_count,total_count,test_day,bucket_date,probe_cc,probe_asn
0,http://www.vibe.com/,0,0,0,1,2019-10-21,2019-10-21,GB,12576
1,http://www.vibe.com/,0,0,0,1,2019-10-21,2019-10-21,GB,35662
2,http://www.vibe.com/,0,0,0,1,2019-10-21,2019-10-21,ID,17974
3,http://www.vibe.com/,0,0,0,1,2019-10-21,2019-10-21,IN,45194
4,http://www.vibe.com/,0,0,0,1,2019-10-21,2019-10-21,IT,137


In [9]:
num_rows = pd.read_sql_query("SELECT COUNT(*) FROM (select * from ooexpl_wc_input_counts limit 10000000) meas", sql_engine)
num_rows

,count
0,10000000


In [10]:
pd.read_sql_query("SELECT distinct to_char(ooexpl_wc_input_counts.test_day, 'YYYY') from ooexpl_wc_input_counts", sql_engine)

,to_char
0,2019


In [11]:
x = 10000000
meas_2019 = pd.read_sql_query("SELECT * from (select * from ooexpl_wc_input_counts limit " + str(x) + ") small_meas where to_char(small_meas.test_day, 'YYYY') = '2018'", sql_engine)
meas_2019.head()

,input,confirmed_count,anomaly_count,failure_count,total_count,test_day,bucket_date,probe_cc,probe_asn


In [12]:
len(meas_2019)

0

In [13]:
x = 10000000
meas_2019_full = meas_2019
count = 1
curr_meas = meas_2019
while len(curr_meas) > 0:
    if count != 1:
        meas_2019_full = meas_2019_full.append(curr_meas)
    curr_meas = pd.read_sql_query("SELECT * from (select * from measurement offset " + str(count * x) + " rows limit " + str(x) + ") small_meas where to_char(small_meas.measurement_start_time, 'YYYY') = '2018'", sql_engine)
    count += 1

In [14]:
meas_2019_full.head()

,input,confirmed_count,anomaly_count,failure_count,total_count,test_day,bucket_date,probe_cc,probe_asn


In [15]:
len(meas_2018_full)

NameError: name 'meas_2018_full' is not defined

In [ ]:
meas_2018_full.to_csv("measurements_2018.csv")

In [ ]:
meas_2018_read = pd.read_csv("measurements_2018.csv")
meas_2018_read.head()

In [ ]:
len(meas_2018_read)

In [ ]:
reports_2018 = pd.read_sql_query("SELECT * from report where to_char(report.test_start_time, 'YYYY') = '2018'", sql_engine)
reports_2018.head()

In [ ]:
len(reports_2018)

In [ ]:
meas_report_join = meas_2018_read.merge(reports_2018, on = "report_no", how = "left")
meas_report_join.head()

In [ ]:
len(meas_report_join)

In [ ]:
meas_report_join.columns

In [ ]:
meas_report_join.groupby("probe_cc").nunique()["report_no"]

In [ ]:
meas_report_join.groupby("confirmed").nunique()["report_no"]

In [ ]:
# count confirmed by country
country_groups = meas_report_join.groupby("probe_cc")
country_groups.first()

In [ ]:
rates_2018 = country_groups.agg({"confirmed": "sum", "anomaly": "sum", "report_no": "count"})
rates_2018.head()

In [ ]:
rates_2018.loc[rates_2018["confirmed"] > 0]

In [ ]:
rates_2018["strict_rate"] = rates_2018['confirmed'] / rates_2018['report_no']
rates_2018["loose_rate"] = (rates_2018['confirmed'] + rates_2018['anomaly']) / rates_2018['report_no']
rates_2018["num_no_confirmed_interference"] = rates_2018["report_no"] - rates_2018['confirmed']

In [ ]:
rates_2018.head()

In [ ]:
rates_2018.drop(['report_no'], axis = 1, inplace = True)
rates_2018.head()

In [ ]:
rates_2018 = rates_2018.rename(index = {"probe_cc": "country_code"}, columns={'confirmed': 'num_confirmed_interference', 'anomaly': "num_anomaly"})

In [ ]:
rates_2018.head()

In [ ]:
rates_2018 = rates_2018.rename_axis("country_code")
rates_2018.head()

In [ ]:
columnsTitles = ["strict_rate", "num_no_confirmed_interference"]
rates_2018 = rates_2018.reindex(columns = columnsTitles)
rates_2018.head()